In [15]:
import numpy as np
import matplotlib.pyplot as plt
rng = np.random.RandomState(1234)

In [16]:
N_pop = int(1e5)  # Population size
N_work = 100      # Number of people in a workplace
N_home = 2        # Number of people in a home

In [17]:
# Static state of the population
v_workplace = np.arange(N_pop) // N_work  # Index of workplace
v_home = np.arange(N_pop) // N_home       # Index of home
rng.shuffle(v_home)                       # Make workplace and home independent

In [5]:
# Initial state of the population - no-one is infected
v_susceptible = np.full(shape=(N_pop,), fill_value=1, dtype=np.float32)
v_infected = np.full(shape=(N_pop,), fill_value=0, dtype=np.float32)
v_days_infected = np.full(shape=(N_pop,), fill_value=0, dtype=np.float32)